# Poker Agent V9: Equity-Aware Hybrid Architecture

This notebook introduces **Explicit Poker Knowledge** into the RL Agent to fix the negative profitability issues of V8.

### The Problem with V8
V8 had a high win rate (>60%) but negative profit per hand. This means it was winning small pots (stealing blinds) but losing big pots or failing to extract value. The Neural Network struggled to learn basic hand strength (e.g., "AA is good") from raw card bits alone.

### V9 Solution: Feature Injection
Instead of forcing the agent to derive probability theory from scratch, we calculate key metrics and feed them as **direct inputs**:

1. **Monte Carlo Equity**: We run 30 simulations per step to estimate win probability (e.g., `0.85`).
2. **Pot Odds**: We calculate the ratio of `Call Cost / Total Pot` (e.g., `0.33`).
3. **Stack-to-Pot Ratio (SPR)**: Critical for decision making.

### Why this helps?
The agent will instantly see correlations like `Equity=0.9` -> `Action=Raise` -> `Reward=$$$`. This "installs" basic poker competence immediately, allowing the RL to focus on the harder part: adapting to opponent styles.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from collections import deque
import random
import matplotlib.pyplot as plt
from typing import Optional, Tuple, List, Dict, Any
import copy
from itertools import combinations

from pokerkit import Automation, NoLimitTexasHoldem, Card, StandardHighHand, Deck

# Constants
SEED = 42
MAX_HISTORY_LEN = 100
ACTION_EMBED_DIM = 16
HIDDEN_DIM_LSTM = 128

# Actions
ENV_FOLD = 0
ENV_CHECK_CALL = 1
ENV_BET_RAISE = 2
NUM_ACTIONS = 3

# History Tokens
ACT_PAD = 0
ACT_V_FOLD = 1
ACT_V_CHECK_CALL = 2
ACT_V_BET_RAISE = 3
OPP_FOLD = 4
OPP_CHECK_CALL = 5
OPP_BET_RAISE = 6
OUT_AGENT_WIN = 7
OUT_AGENT_LOSS = 8
OUT_TIE = 9
OUT_NEW_HAND = 10
HISTORY_VOCAB_SIZE = 11

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Universal Helpers (Monte Carlo & Flatten)

def flatten_cards_list(items):
    out = []
    if isinstance(items, Card): return [items]
    for x in items:
        if isinstance(x, (list, tuple)): out.extend(flatten_cards_list(x))
        else: out.append(x)
    return out

def monte_carlo_equity(hole_cards: List[Card], board_cards: List[Card], iterations=30) -> float:
    """Run fast MC simulation to estimate win probability."""
    if not hole_cards: return 0.5 
    
    wins = 0
    # Flatten inputs just in case
    hole_cards = flatten_cards_list(hole_cards)
    board_cards = flatten_cards_list(board_cards)
    known_cards = set(hole_cards + board_cards)
    
    for _ in range(iterations):
        deck_cards = [c for c in Deck.STANDARD if c not in known_cards]
        random.shuffle(deck_cards)
        
        # Opponent hand (2 cards)
        opp_hole = deck_cards[:2]
        
        # Remaining board
        needed_board = 5 - len(board_cards)
        sim_board = board_cards + deck_cards[2:2+needed_board]
        
        my_total = hole_cards + sim_board
        opp_total = opp_hole + sim_board
        
        # Best 5 of 7
        my_hand = max(StandardHighHand(c) for c in combinations(my_total, 5))
        opp_hand = max(StandardHighHand(c) for c in combinations(opp_total, 5))
        
        if my_hand > opp_hand: wins += 1
        elif my_hand == opp_hand: wins += 0.5
            
    return wins / iterations

In [ ]:
class EquityPokerEnv(gym.Env):
    """
    V9 Environment: Adds Equity, PotOdds, and SPR to the observation vector.
    """
    def __init__(self, num_players: int = 2, starting_stack: int = 1000, 
                 small_blind: int = 5, big_blind: int = 10):
        super().__init__()
        self.num_players = num_players
        self.starting_stack = starting_stack
        self.small_blind = small_blind
        self.big_blind = big_blind
        
        # Base dim (from V8/V7) was ~320. We add 3 floats: Equity, PotOdds, SPR.
        self.base_state_dim = 52*2 + 52*5 + num_players + 1 + 1 + 4 + 1
        self.game_state_dim = self.base_state_dim + 3 
        
        self.observation_space = spaces.Dict({
            'game_state': spaces.Box(low=0, high=1, shape=(self.game_state_dim,), dtype=np.float32),
            'history': spaces.Box(low=0, high=HISTORY_VOCAB_SIZE-1, shape=(MAX_HISTORY_LEN,), dtype=np.int64)
        })
        self.action_space = spaces.Discrete(NUM_ACTIONS)
        self.state = None
        self.agent_player_index = 0
        self.global_history = deque(maxlen=MAX_HISTORY_LEN)
        for _ in range(MAX_HISTORY_LEN): self.global_history.append(ACT_PAD)
        
    # --- Card Encoding Helpers (Same as V8) ---
    def _card_to_index(self, card: Card) -> int:
        ranks = '23456789TJQKA'
        suits = 'cdhs'
        rank_idx = ranks.index(card.rank)
        suit_idx = suits.index(card.suit)
        return rank_idx * 4 + suit_idx
    
    def _encode_card(self, card: Optional[Card]) -> np.ndarray:
        encoding = np.zeros(52, dtype=np.float32)
        if card is not None: encoding[self._card_to_index(card)] = 1.0
        return encoding
    
    # --- Observation with Feature Injection ---
    def _get_observation(self) -> Dict[str, Any]:
        # 1. Feature Calculation
        hole = flatten_cards_list(self.state.hole_cards[self.agent_player_index])
        board = flatten_cards_list(self.state.board_cards)
        
        # A. Monte Carlo Equity
        equity = monte_carlo_equity(hole, board, iterations=25)
        
        # B. Pot Odds
        total_pot = sum(self.state.bets)
        current_bet = max(self.state.bets)
        my_bet = self.state.bets[self.agent_player_index]
        to_call = current_bet - my_bet
        pot_odds = 0.0
        if (total_pot + to_call) > 0:
            pot_odds = to_call / (total_pot + to_call)
            
        # C. SPR (Stack to Pot Ratio)
        my_stack = self.state.stacks[self.agent_player_index]
        spr = 0.0
        if total_pot > 0:
            spr = my_stack / total_pot
        # Normalize SPR (clamp at 20)
        spr = min(spr / 20.0, 1.0)
        
        # 2. Build State Vector
        state_vector = [equity, pot_odds, spr]
        
        # Append Standard Card/Board Vector (Same as V8)
        for i in range(2):
            if i < len(hole): state_vector.extend(self._encode_card(hole[i]))
            else: state_vector.extend(np.zeros(52, dtype=np.float32))
        for i in range(5):
            if i < len(board): state_vector.extend(self._encode_card(board[i]))
            else: state_vector.extend(np.zeros(52, dtype=np.float32))
        for i in range(self.num_players):
            stack = self.state.stacks[i] / self.starting_stack
            state_vector.append(min(stack, 2.0))
        state_vector.append(total_pot / (self.starting_stack * self.num_players))
        if self.state.actor_index is not None:
            state_vector.append(self.state.actor_index / max(1, self.num_players - 1))
        else: state_vector.append(0.0)
        street = [0.0, 0.0, 0.0, 0.0]
        num_board = len(board)
        if num_board == 0: street[0] = 1.0
        elif num_board == 3: street[1] = 1.0
        elif num_board == 4: street[2] = 1.0
        else: street[3] = 1.0
        state_vector.extend(street)
        state_vector.append(float(self.agent_player_index))
        
        return {
            'game_state': np.array(state_vector, dtype=np.float32),
            'history': np.array(list(self.global_history), dtype=np.int64)
        }
    
    # --- Env Logic (V8 Standard) ---
    def _update_history(self, player_idx: int, action: int):
        if player_idx == self.agent_player_index:
            if action == ENV_FOLD: token = ACT_V_FOLD
            elif action == ENV_CHECK_CALL: token = ACT_V_CHECK_CALL
            else: token = ACT_V_BET_RAISE
        else:
            if action == ENV_FOLD: token = OPP_FOLD
            elif action == ENV_CHECK_CALL: token = OPP_CHECK_CALL
            else: token = OPP_BET_RAISE
        self.global_history.append(token)

    def append_outcome_token(self, final_reward: float):
        if final_reward > 0: self.global_history.append(OUT_AGENT_WIN)
        elif final_reward < 0: self.global_history.append(OUT_AGENT_LOSS)
        else: self.global_history.append(OUT_TIE)

    def _get_legal_actions(self) -> List[int]:
        legal = []
        if self.state.can_fold(): legal.append(ENV_FOLD)
        if self.state.can_check_or_call(): legal.append(ENV_CHECK_CALL)
        if self.state.can_complete_bet_or_raise_to(): legal.append(ENV_BET_RAISE)
        return legal if legal else [ENV_CHECK_CALL]
    
    def _execute_action(self, action: int) -> None:
        if action == ENV_FOLD:
            if self.state.can_fold(): self.state.fold()
            elif self.state.can_check_or_call(): self.state.check_or_call()
        elif action == ENV_CHECK_CALL:
            if self.state.can_check_or_call(): self.state.check_or_call()
            elif self.state.can_fold(): self.state.fold()
        elif action == ENV_BET_RAISE:
            if self.state.can_complete_bet_or_raise_to():
                min_r = self.state.min_completion_betting_or_raising_to_amount
                max_r = self.state.max_completion_betting_or_raising_to_amount
                self.state.complete_bet_or_raise_to(min(min_r * 2, max_r))
            elif self.state.can_check_or_call():
                self.state.check_or_call()
    
    def _run_automations(self) -> None:
        while self.state.can_burn_card(): self.state.burn_card('??')
        while self.state.can_deal_board(): self.state.deal_board()
        while self.state.can_push_chips(): self.state.push_chips()
        while self.state.can_pull_chips(): self.state.pull_chips()
    
    def reset(self, seed=None, options=None) -> Tuple[Dict, Dict]:
        self.global_history.append(OUT_NEW_HAND)
        # Use super().reset() but we wrap basic pokerkit init manually
        # gym.Env.reset(self, seed=seed) # Fixed super call
        self.state = NoLimitTexasHoldem.create_state(
            automations=(Automation.ANTE_POSTING, Automation.BET_COLLECTION, Automation.BLIND_OR_STRADDLE_POSTING, Automation.HOLE_CARDS_SHOWING_OR_MUCKING, Automation.HAND_KILLING, Automation.CHIPS_PUSHING, Automation.CHIPS_PULLING),
            ante_trimming_status=True,
            raw_antes={-1: 0},
            raw_blinds_or_straddles=(self.small_blind, self.big_blind),
            min_bet=self.big_blind,
            raw_starting_stacks=[self.starting_stack] * self.num_players,
            player_count=self.num_players,
        )
        while self.state.can_deal_hole(): self.state.deal_hole()
        self._run_automations()
        return self._get_observation(), {'legal_actions': self._get_legal_actions()}
    
    def step(self, action: int) -> Tuple[Dict, float, bool, bool, Dict]:
        if self.state.actor_index is not None:
             self._update_history(self.state.actor_index, action)
        self._execute_action(action)
        self._run_automations()
        done = self.state.status is False
        reward = 0.0
        if done:
            reward = (self.state.stacks[self.agent_player_index] - self.starting_stack) / self.big_blind
        obs = self._get_observation()
        info = {'legal_actions': self._get_legal_actions() if not done else []}
        return obs, reward, done, False, info
    
    def get_final_reward(self) -> float:
        return (self.state.stacks[self.agent_player_index] - self.starting_stack) / self.big_blind
    
    def update_opponent_history(self, action: int):
        opp_idx = 1 - self.agent_player_index
        self._update_history(opp_idx, action)

In [ ]:
class FeatureAwareV9(nn.Module):
    def __init__(self, state_dim: int, action_dim: int):
        super().__init__()
        # State Branch (Including new Features)
        self.state_net = nn.Sequential(
            nn.Linear(state_dim, 256), nn.LayerNorm(256), nn.ReLU(),
            nn.Linear(256, 128), nn.ReLU()
        )
        # History Branch
        self.action_embedding = nn.Embedding(HISTORY_VOCAB_SIZE, ACTION_EMBED_DIM)
        self.lstm = nn.LSTM(input_size=ACTION_EMBED_DIM, hidden_size=HIDDEN_DIM_LSTM, batch_first=True)
        
        # Merger
        self.value_head = nn.Sequential(
            nn.Linear(128 + HIDDEN_DIM_LSTM, 256), nn.ReLU(),
            nn.Linear(256, action_dim)
        )
        
    def forward(self, state, history):
        s_feat = self.state_net(state)
        h_embed = self.action_embedding(history)
        lstm_out, (hn, cn) = self.lstm(h_embed)
        h_context = hn[-1]
        combined = torch.cat([s_feat, h_context], dim=1)
        return self.value_head(combined)

class ReplayBufferV9:
    def __init__(self, capacity=50000):
        self.buffer = deque(maxlen=capacity)
    def push(self, transition):
        self.buffer.append(transition)
    def sample(self, batch_size):
        return random.sample(self.buffer, min(len(self.buffer), batch_size))
    def __len__(self): return len(self.buffer)

class HybridAgentV9:
    def __init__(self, state_dim, action_dim=NUM_ACTIONS, lr=1e-4):
        self.model = FeatureAwareV9(state_dim, action_dim).to(device)
        self.target_model = FeatureAwareV9(state_dim, action_dim).to(device)
        self.target_model.load_state_dict(self.model.state_dict())
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.99995
        
    def select_action(self, obs, legal_actions, eval_mode=False):
        if not eval_mode and random.random() < self.epsilon:
            return random.choice(legal_actions)
        state_t = torch.FloatTensor(obs['game_state']).unsqueeze(0).to(device)
        h_t = torch.LongTensor(obs['history']).unsqueeze(0).to(device)
        with torch.no_grad():
            q_values = self.model(state_t, h_t)
        q_numpy = q_values.cpu().numpy().flatten()
        masked_q = np.full(NUM_ACTIONS, -np.inf)
        for a in legal_actions: masked_q[a] = q_numpy[a]
        return int(np.argmax(masked_q))

    def train(self, buffer, batch_size=64):
        if len(buffer) < batch_size: return None
        batch = buffer.sample(batch_size)
        # Batch unpacking
        states = torch.FloatTensor(np.array([t[0] for t in batch])).to(device)
        histories = torch.LongTensor(np.array([t[1] for t in batch])).to(device)
        actions = torch.LongTensor(np.array([t[2] for t in batch])).to(device)
        rewards = torch.FloatTensor(np.array([t[3] for t in batch])).to(device)
        next_states = torch.FloatTensor(np.array([t[4] for t in batch])).to(device)
        next_histories = torch.LongTensor(np.array([t[5] for t in batch])).to(device)
        dones = torch.FloatTensor(np.array([t[6] for t in batch])).to(device)
        
        # DQN Logic
        current_q = self.model(states, histories).gather(1, actions.unsqueeze(1)).squeeze(1)
        with torch.no_grad():
            next_actions = self.model(next_states, next_histories).argmax(1).unsqueeze(1)
            target_q_next = self.target_model(next_states, next_histories).gather(1, next_actions).squeeze(1)
            target = rewards + (1 - dones) * self.gamma * target_q_next
        loss = F.mse_loss(current_q, target)
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
        self.optimizer.step()
        return loss.item()
    
    def update_target(self): self.target_model.load_state_dict(self.model.state_dict())

In [ ]:
# Copy Bots from V8 (ValueBot, BluffBot, BalancedBot)
# Ensure they use the robust flatten_cards_list logic too
class HeuristicBot:
    def __init__(self, player_idx=1):
        self.player_idx = player_idx
        
    def get_equity(self, state):
        # Use V9 robust evaluator
        hole = flatten_cards_list(state.hole_cards[self.player_idx])
        board = flatten_cards_list(state.board_cards)
        return monte_carlo_equity(hole, board, iterations=40)
        
    def select_action(self, state, legal_actions):
        raise NotImplementedError

class ValueBot(HeuristicBot):
    def select_action(self, state, legal_actions):
        equity = self.get_equity(state)
        if equity > 0.70 and ENV_BET_RAISE in legal_actions: return ENV_BET_RAISE
        if equity > 0.45 and ENV_CHECK_CALL in legal_actions: return ENV_CHECK_CALL
        if ENV_FOLD in legal_actions: return ENV_FOLD
        return ENV_CHECK_CALL

class BluffBot(HeuristicBot):
    def select_action(self, state, legal_actions):
        equity = self.get_equity(state)
        if equity > 0.70 and ENV_BET_RAISE in legal_actions: return ENV_BET_RAISE
        if equity < 0.40 and random.random() < 0.30 and ENV_BET_RAISE in legal_actions: return ENV_BET_RAISE
        if equity > 0.45 and ENV_CHECK_CALL in legal_actions: return ENV_CHECK_CALL
        if ENV_FOLD in legal_actions: return ENV_FOLD
        return ENV_CHECK_CALL

class BalancedBot(HeuristicBot):
    def select_action(self, state, legal_actions):
        equity = self.get_equity(state)
        if equity > 0.8 and ENV_BET_RAISE in legal_actions: return ENV_BET_RAISE
        if ENV_CHECK_CALL in legal_actions:
            if equity > 0.5: return ENV_CHECK_CALL
            current_bet = max(state.bets)
            my_bet = state.bets[self.player_idx]
            to_call = current_bet - my_bet
            pot_odds = to_call / (sum(state.bets) + to_call + 1e-5)
            if equity > pot_odds: return ENV_CHECK_CALL
        if ENV_FOLD in legal_actions: return ENV_FOLD
        return ENV_CHECK_CALL

# Standard Bots
class ManiacAgent:
    def select_action(self, state, legal_actions):
        if ENV_BET_RAISE in legal_actions: return ENV_BET_RAISE
        if ENV_CHECK_CALL in legal_actions: return ENV_CHECK_CALL
        return ENV_FOLD
class NitAgent:
    def select_action(self, state, legal_actions):
        if ENV_FOLD in legal_actions and ENV_CHECK_CALL in legal_actions:
            if random.random() < 0.9: return ENV_FOLD
        if ENV_CHECK_CALL in legal_actions: return ENV_CHECK_CALL
        return ENV_FOLD
class RandomAgent:
    def select_action(self, state, legal_actions): return random.choice(legal_actions)

In [ ]:
def train_v9(num_hands=50000):
    env = EquityPokerEnv()
    agent = HybridAgentV9(env.game_state_dim)
    buffer = ReplayBufferV9(capacity=50000)
    
    opps = {
        'Maniac': ManiacAgent(),
        'Nit': NitAgent(),
        'Random': RandomAgent(),
        'ValueBot': ValueBot(),
        'BluffBot': BluffBot(),
        'Balanced': BalancedBot()
    }
    opp_names = list(opps.keys())
    stats = {name: {'rewards': [], 'wins': 0, 'hands': 0} for name in opp_names}
    
    print(f"Training V9 (Equity-Hybrid) for {num_hands} hands...")
    current_opp_name = 'Random'
    
    for hand in range(num_hands):
        if hand % 20 == 0: current_opp_name = random.choice(opp_names)
        opponent = opps[current_opp_name]
        
        obs, info = env.reset()
        done = False
        episode_transitions = []
        pending_agent_obs = None
        pending_agent_action = None
        
        while not done:
            if env.state.actor_index == env.agent_player_index:
                if pending_agent_obs is not None:
                    episode_transitions.append((pending_agent_obs['game_state'], pending_agent_obs['history'], pending_agent_action, 0.0, obs['game_state'], obs['history'], False, info['legal_actions']))
                
                action = agent.select_action(obs, info['legal_actions'])
                pending_agent_obs = obs
                pending_agent_action = action
                obs, reward, done, _, info = env.step(action)
            else:
                action = opponent.select_action(env.state, info['legal_actions'])
                env.update_opponent_history(action)
                env._execute_action(action)
                env._run_automations()
                done = env.state.status is False
                if not done:
                    obs = env._get_observation()
                    info['legal_actions'] = env._get_legal_actions()
        
        final_reward = env.get_final_reward()
        env.append_outcome_token(final_reward)
        term_obs = env._get_observation()
        
        if pending_agent_obs is not None:
             episode_transitions.append((pending_agent_obs['game_state'], pending_agent_obs['history'], pending_agent_action, 0.0, term_obs['game_state'], term_obs['history'], True, []))
        
        stats[current_opp_name]['rewards'].append(final_reward)
        stats[current_opp_name]['hands'] += 1
        if final_reward > 0: stats[current_opp_name]['wins'] += 1
        
        for i in range(len(episode_transitions)):
            s, h, a, r, ns, nh, d, l = episode_transitions[i]
            buffer.push((s, h, a, final_reward, ns, nh, d, l))
            
        if len(buffer) > 1000:
            agent.train(buffer)
        agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)
        if hand % 500 == 0: agent.update_target()
        
        if hand % 2500 == 0 and hand > 0:
            print(f"\n=== Checkpoint Hand {hand} (Eps: {agent.epsilon:.2f}) ===")
            print("--- Extreme Opponents ---")
            for name in ['Maniac', 'Nit', 'Random']:
                 if stats[name]['hands'] > 0:
                     print(f"{name}: Avg {np.mean(stats[name]['rewards'][-200:]):.2f} BB | Win {stats[name]['wins']/stats[name]['hands']:.1%}")
            print("--- Realistic Opponents ---")
            for name in ['ValueBot', 'BluffBot', 'Balanced']:
                 if stats[name]['hands'] > 0:
                     print(f"{name}: Avg {np.mean(stats[name]['rewards'][-200:]):.2f} BB | Win {stats[name]['wins']/stats[name]['hands']:.1%}")
    return agent, stats

def plot_v9_results(stats):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    for name in ['Maniac', 'Nit', 'Random']:
        if len(stats[name]['rewards']) > 100:
            ax1.plot(np.convolve(stats[name]['rewards'], np.ones(100)/100, mode='valid'), label=name)
    ax1.set_title("V9 vs Extreme Bots")
    ax1.legend()
    for name in ['ValueBot', 'BluffBot', 'Balanced']:
        if len(stats[name]['rewards']) > 100:
            ax2.plot(np.convolve(stats[name]['rewards'], np.ones(100)/100, mode='valid'), label=name)
    ax2.set_title("V9 vs Realistic Bots")
    ax2.legend()
    plt.show()
    plt.savefig("v9_results.png")

agent, stats = train_v9(50000)
plot_v9_results(stats)

In [8]:
# Final Evaluation Mode (Zero Noise)
def evaluate_agent(agent, num_hands_per_opp=1000):
    print(f"\n=== FINAL EVALUATION (Epsilon=0.0) ===")
    # Force pure exploitation
    agent.epsilon = 0.0
    env = EquityPokerEnv()
    
    opps = {
        'Maniac': ManiacAgent(),
        'Nit': NitAgent(),
        'Random': RandomAgent(),
        'ValueBot': ValueBot(),
        'BluffBot': BluffBot(),
        'Balanced': BalancedBot()
    }
    
    # We want to give the LSTM time to adapt, so we play a contiguous block
    for name, opponent in opps.items():
        rewards = []
        wins = 0
        
        # Reset History for new opponent to give clean slate (optional choice)
        env.global_history.clear()
        for _ in range(MAX_HISTORY_LEN): env.global_history.append(ACT_PAD)
        
        for _ in range(num_hands_per_opp):
            obs, info = env.reset()
            done = False
            while not done:
                if env.state.actor_index == env.agent_player_index:
                    action = agent.select_action(obs, info['legal_actions'])
                    obs, _, done, _, info = env.step(action)
                else:
                    action = opponent.select_action(env.state, info['legal_actions'])
                    env.update_opponent_history(action)
                    env._execute_action(action)
                    env._run_automations()
                    done = env.state.status is False
                    if not done:
                        obs = env._get_observation()
                        info['legal_actions'] = env._get_legal_actions()
            
            final_reward = env.get_final_reward()
            env.append_outcome_token(final_reward)
            rewards.append(final_reward)
            if final_reward > 0: wins += 1
            
        avg_score = np.mean(rewards)
        win_rate = wins / num_hands_per_opp
        print(f"Vs {name}: Avg {avg_score:.2f} BB/hand | Win Rate {win_rate:.1%} | Total Profit: {sum(rewards):.1f} BB")

evaluate_agent(agent, 3000)


=== FINAL EVALUATION (Epsilon=0.0) ===
Vs Maniac: Avg -3.55 BB/hand | Win Rate 0.4% | Total Profit: -10652.0 BB
Vs Nit: Avg 0.47 BB/hand | Win Rate 95.6% | Total Profit: 1402.5 BB
Vs Random: Avg -0.72 BB/hand | Win Rate 43.6% | Total Profit: -2159.0 BB
Vs ValueBot: Avg -0.75 BB/hand | Win Rate 62.6% | Total Profit: -2248.0 BB
Vs BluffBot: Avg -0.86 BB/hand | Win Rate 52.7% | Total Profit: -2572.5 BB
Vs Balanced: Avg -1.11 BB/hand | Win Rate 49.1% | Total Profit: -3331.5 BB
